In [154]:
# Importando os pacotes que serão utilizados

# Para manipulação e tratamento dos dados
import numpy as np
import pandas as pd 
import time #utilizada para funções de tempo
import matplotlib.pyplot as plt #Utilizada para gráficos
import seaborn as sns #Utilizada para gráficos

# Bibliotecas do Skit Learn
from sklearn.model_selection import train_test_split #Utilizada para separar dados pra treino e teste
from sklearn.preprocessing import StandardScaler #Utilizada para fazer a padronização dos dados
from sklearn.preprocessing import LabelEncoder #Utilizada para fazer o OneHotEncoding
from sklearn.metrics import accuracy_score #Utilizada para avaliar a acurácia do modelo preditivo
from sklearn.neighbors import KNeighborsClassifier #Nosso Algoritmo para criação do modelo
from imblearn import under_sampling, over_sampling #Utilizada para fazer o balanceamento de dados
from imblearn.over_sampling import SMOTE #Utilizada para fazer o balanceamento de dados


# Para remover avisos de alerta
import warnings #Utilizada para avisos de alertas
warnings.filterwarnings("ignore") #Ignorar avisos de alertas. Obs.: Alertas NÃO são erros, são apenas mensagens informativas

# Para não limitar a exibição do DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format

In [155]:
df_pedido = pd.read_csv('data/03013604.csv', encoding='latin1', sep=';')
df_clientes = pd.read_csv('data/0105070402.csv', encoding='latin1', sep=';')
df_devolvidos = pd.read_csv('data/030237.csv', encoding='latin1', sep=';')

In [156]:
df_pedido.columns = df_pedido.columns.str.strip()
df_devolvidos.columns = df_devolvidos.columns.str.strip()
df_clientes.columns = df_clientes.columns.str.strip()

In [157]:
df_pedido_val = df_pedido.drop(columns=[
  'UNB', 
  'UNB Origem', 
  'Nome PDV', 
  'Alcada Anterior 1',
  'Alcada Anterior 2',
  'Alcada Anterior 3',
  'Alcada Atual 1',
  'Alcada Atual 2',
  'Alcada Atual 3',
  'Aprovador 1',
  'Decisao Alcada 1',
  'Aprovador 2',
  'Decisao Alcada 2',
  'Aprovador 3',
  'Decisao Alcada 3',
  'Critica 1',
  'Critica 2',
  'Critica 3',
  'Critica 4',
  'Critica 5',
  'Critica 6',
  'Alcada Rejeicao',
  'Critica Rejeicao',
  'Preco S/ ADF',
  'Minimo Politica',
  'TTV Planejado',
  'Valor Sugerido',
  'Tabela Escalonada',
  'M1',
  'M2',
  'M3',
  'M2/M3',
  'Segmentada',
  'Telemarketing',
  'Empresa Destino',
  'Filial Destino',
  'Justificativa Alcada 1',
  'Justificativa Alcada 2',
  'Justificativa Alcada 3',
  'Embalagem',
  'Unid. Venda Produto',
  'Fator Conversão Pedido',
  'Unid. Venda PGV',
  'Fator Conversão PGV',
  'Embalagem SIV',
  'Crítica LC Vendas',
  'St. Fat. Automático',
  'Vendedor GCAD',
  'Cod. Setor Atendimento',
  'Tipo Setor Atendimento',
  'Situação',
  'Idade',
  'Pedido Quebra Original',
  'GUID Entrega Hércules',
  'GUID Item Hércules',
  'Desc Disponível Padrão',
  'Desc Disponível Ação',
  'Desconto Algoritmo Aut',
  ''
  ])

In [158]:
df_pedido_val['Status Pedido'] = df_pedido_val['Status Pedido'].astype(str)
df_pedido_val['Unid. Venda'] = df_pedido_val['Unid. Venda'].astype(str)
df_pedido_val['Ocorrencia'] = df_pedido_val['Ocorrencia'].astype(str)
df_pedido_val['Origem Pedido'] = df_pedido_val['Origem Pedido'].astype(str)
df_pedido_val['Fora de Rota'] = df_pedido_val['Fora de Rota'].astype(str)
df_pedido_val['Desconto na Rota'] = df_pedido_val['Desconto na Rota'].astype(str)
df_pedido_val['Desconto Fora de Rota'] = df_pedido_val['Desconto Fora de Rota'].astype(str)
df_pedido_val['Pedido Exceção'] = df_pedido_val['Pedido Exceção'].astype(str)


In [159]:
def tratar_float(valor):
    if isinstance(valor, str):
      valor = valor.strip()
    
      if valor == '':
          return np.nan
      valor = valor.replace('.', '').replace(',', '.')
    
    return valor

for i in df_pedido_val.columns.to_list():
  df_pedido_val[i] = df_pedido_val[i].apply(tratar_float)

In [160]:
df_pedido_val['Total Pedido'] = df_pedido_val['Total Pedido'].astype(float)
df_pedido_val['Total Cliente'] = df_pedido_val['Total Cliente'].astype(float)
df_pedido_val['Preco Unitario'] = df_pedido_val['Preco Unitario'].astype(float)
df_pedido_val['Compra Media'] = df_pedido_val['Compra Media'].astype(float)
df_pedido_val['Compra Media Ajustada'] = df_pedido_val['Compra Media Ajustada'].astype(float)
df_pedido_val['Volume Pedido'] = df_pedido_val['Volume Pedido'].astype(float)
df_pedido_val['Volume Total'] = df_pedido_val['Volume Total'].astype(float)


In [161]:
df_clientes_var = df_clientes.drop(columns=[
    'Empresa',
    'Filial',
    'Documento',
    'Nome Fantasia',
    'Razão Social',
    'Endereço',
    'Complemento',
    'UF',
    'Telefone(s)',
    'Cód Rede',
    'Nome Rede',
    'Modelo Score',
    'Segmento Score',
    'Limite de Crédito',
    'Limite de Crédito Usado',
    'Limite de Crédito Disponível',
    'Prazo Máximo Matriz',
    'Maior Prazo por Título',
    'Perc Pgto em Atraso',
    'Situação Documentação',
    'Data de Cadastro',
    'Data da Última Compra',
    'Compra Média',
    'Maior Compra',
    'Acumulado de Vendas',
    'Títulos Pendentes',
    'Títulos à Vencer',
    'Perc CR à Vencer (M-1)',
    'Data de Envio Serasa',
    'Data de Retorno Serasa',
    'Cód EG',
    'Cliente em Inativação',
    'Motivo Lista de Restrições',
    'Tipo de Pessoa',
    'Setor VDE',
    'Área VDE',
    'GV VDE',
    'Data Início da Visita VDE',
    'Dia de Visita do VDE',
    'Data da Prox Visita VDE',
    'Perc Positivação VDE(Mês)',
    'Perc Positivação VDE(TRI)',
    'Perc GPS VDE(Mês)',
    'Setor VDI',
    'Área VDI',
    'GV VDI',
    'Data Início da Visita VDI',
    'Dia de Visita do VDI',
    'Data da Prox Visita VDI',
    'Perc Positivação VDI(Mês)',
    'Perc Positivação VDI(TRI)',
    'Perc Contatos VDI(Mês)',
    'Prazo Médio Ponderado à Prazo (M-1)',
    'Prazo Médio Ponderado (M-1)',
    'Faturamento à Prazo (M-1)',
    'Faturamento DH (M-1)',
    'Faturamento Total (M-1)',
    'CR a Vencer (M-1)',
    'Prazo Médio Ponderado à Prazo (Mês)',
    'Prazo Médio Ponderado (Mês)',
    'Faturamento à Prazo (Mês)',
    'Faturamento DH (Mês)',
    'Faturamento Total (Mês)',
    'CR a Vencer (Mês)',
    'Perc CR a Vencer (Mês)',
    'Perc Atraso(6 meses)',
    'Maior Atraso(6 meses)',
    'Média Faturamento(3 meses)',
    'Segmento NGE',
    'Unnamed: 81',
    'Cond Pag Atual',
    'ADF',
    'Saldo em Aberto',
    'Maior Prazo Histórico',
    'Vlr Total Protestos',
    'Qtde Ações Judiciais',
    'Vlr Total Ações Judiciais',
    'Qtde Falências/Concordatas/Rec Judicial',
    'Qtde Pend Bancárias',
    'Vlr Total Pend Bancárias',
    'Qtde Part. Insucesso Empresarial',
    'Qtde Cheques Sem Fundos/Reapresentados',
    'Vlr Total Cheques Sem Fundos/Reapresentados',
    'Qtde Pend Comerciais',
    'Vlr Total Pend Comerciais',
    'Qtde Dívidas Vencidas',
    'Vlr Total Dívidas Vencidas',
    'Elegível à Extensão de Prazo',
    'Data Última Alteração Elegível',
    'Hora Última Alteração Elegível',
    'Origem Alteração Elegível',
    'Prazo Máximo Elegível',
    'Taxa Extensão de Prazo Elegível',
    'Enviar FIDC Elegível',
    'Novo Prazo Mínimo Elegível',
    'Lucro Real',
    'Presumido',
    'Simples Nacional',
    'Pessoa Física',
    'Consumidor Final',
    'Não Aplica Diferimento',
    'Não Aplica Redução ICMS',
    'Usuário Alteração Elegível',
    'Títulos Abertos FIDC',
    'Limite Crédito Total (Lim Cliente + Títulos Abertos FIDC)',
    'Limite Total Usado (Títulos Abertos + Títulos Abertos FIDC)',
    'Limite Total Disponível (Lim Total - Lim Total Usado)',
    'Perc Atraso FIDC(6 meses)',
    'Perc Atraso FIDC + AMBEV(6 meses)',
    'Unnamed: 123',
    'Situação na RF',
    'CEP'
])

In [162]:
df_clientes_var.rename(columns={'Cód PDV': 'Cod. PDV'}, inplace=True)

In [163]:
df_devolvidos['Nr. Pedido'] = df_devolvidos['Nr. Pedido'].str.strip()
df_devolvidos_notnull = df_devolvidos[df_devolvidos['Nr. Pedido'] != '']

In [164]:
columns = [
    'Nr. Pedido',
    'Status'
]

df_devolvidos_var = df_devolvidos_notnull[columns]

In [165]:
df_devolvidos_var.rename(columns={'Nr. Pedido': 'Pedido'}, inplace=True)

In [166]:
df_devolvidos_var.isna().sum()

Pedido    0
Status    0
dtype: int64

In [167]:
df_devolvidos_var.head()

,Pedido,Status
5,946962,A
6,946919,A
7,946925,A
8,946934,A
9,946716,A


In [168]:
df_pedido_cliente = pd.merge(df_pedido_val, df_clientes_var, on='Cod. PDV', how='left') 

In [169]:
df_pedido_cliente.head()

,Pedido,Data Pedido,Operacao,Cod. PDV,Setor,Status Pedido,Total Pedido,Total Cliente,Produto,Quantidade,Unid. Venda,Preco Unitario,Ocorrencia,Compra Media,Compra Media Ajustada,Volume Pedido,Volume Total,Data Liberacao,Hora Liberacao,Codigo GV,Codigo PGV,Tipo Movimento,Origem Pedido,Desconto Algoritmo,Meta Algoritmo,Código Ação Desconto Alg Fix,Fora de Rota,Desconto na Rota,Desconto Fora de Rota,Pedido Exceção,Bairro,Cidade,Categoria,Classe Risco Atual,Pontuação Score,Prazo Atual,Atraso Médio,Possui Comodato,Status do PDV,Qtde Protestos,Negativado
0,970056,01/10/2024,1,12941,112,Rejeitado,2006.02,0.00,988,20,Dz,73.25,NaN,NaN,NaN,13.40,13.40,01/10/2024,18:01,1,988,51,DER,0,Não,NaN,S,S,N,N,CONJUNTO MIRAGE ...,ANAPOLIS,REST./PIZZARIA/CHURR,3,87,0,0.00,Sim,Ativo,0,N
1,970056,01/10/2024,1,12941,112,Rejeitado,2006.02,0.00,2546,6,Dz,84.95,NaN,NaN,NaN,13.40,13.40,01/10/2024,18:01,1,2546,51,DER,0,Não,NaN,S,S,N,N,CONJUNTO MIRAGE ...,ANAPOLIS,REST./PIZZARIA/CHURR,3,87,0,0.00,Sim,Ativo,0,N
2,970056,01/10/2024,1,12941,112,Rejeitado,2006.02,0.00,503,1,cx,31.32,NaN,NaN,NaN,13.40,13.40,01/10/2024,18:01,1,503,51,DER,0,Não,NaN,S,S,N,N,CONJUNTO MIRAGE ...,ANAPOLIS,REST./PIZZARIA/CHURR,3,87,0,0.00,Sim,Ativo,0,N
3,970118,01/10/2024,1,17969,351,Rejeitado,11592.52,0.00,9071,20,cx,47.92,-->Falta,NaN,NaN,95.90,95.90,01/10/2024,18:01,3,7733,51,DER,0,Não,NaN,S,S,N,N,VILA SANTA MARIA DE NAZARETH ...,ANAPOLIS,ATACADISTA,2,91,10,0.00,Nao,Ativo,0,N
4,970118,01/10/2024,1,17969,351,Rejeitado,11592.52,0.00,504,20,cx,34.65,NaN,NaN,NaN,95.90,95.90,01/10/2024,18:01,3,504,51,DER,0,Não,NaN,S,S,N,N,VILA SANTA MARIA DE NAZARETH ...,ANAPOLIS,ATACADISTA,2,91,10,0.00,Nao,Ativo,0,N


In [170]:
df_pedido_cliente['Pedido'] = df_pedido_cliente['Pedido'].astype(str)
df_devolvidos_var['Pedido'] = df_devolvidos_var['Pedido'].astype(str)
data = pd.merge(df_pedido_cliente, df_devolvidos_var, on='Pedido', how='left')

In [171]:
data.shape

(255992, 42)

In [172]:
data.isna().sum()
data_v2 = data[data['Status'].notna()]

In [173]:
data_v2.shape

(239060, 42)

In [174]:
data_v3 = data_v2.drop(columns=[
 'Ocorrencia',
 'Compra Media',
 'Compra Media Ajustada',
 'Volume Pedido',
 'Volume Total',
 'Código Ação Desconto Alg Fix',
 'Data Pedido',
 'Pedido',
 'Status Pedido',
 'Data Liberacao',
 'Hora Liberacao',
 'Total Cliente',
 'Desconto Algoritmo',
 'Meta Algoritmo',
 'Cod. PDV',
 'Quantidade',
 'Unid. Venda',
 'Bairro',
 'Preco Unitario',
 'Codigo PGV',
 'Qtde Protestos'
 ])

In [175]:
data_v3.head()

,Operacao,Setor,Total Pedido,Produto,Codigo GV,Tipo Movimento,Origem Pedido,Fora de Rota,Desconto na Rota,Desconto Fora de Rota,Pedido Exceção,Cidade,Categoria,Classe Risco Atual,Pontuação Score,Prazo Atual,Atraso Médio,Possui Comodato,Status do PDV,Negativado,Status
23,1,206,722.60,1898,2,51,B2BG,N,N,N,N,ABADIANIA,DEPOSITO DE BEBIDAS,5,61,0,6.00,Nao,Ativo,N,A
51,1,108,875.60,9092,3,51,B2BGA,N,N,N,N,ANAPOLIS,MINIMERCADO,3,86,13,0.00,Sim,Ativo,N,A
52,1,108,875.60,13065,3,51,B2BGA,N,N,N,N,ANAPOLIS,MINIMERCADO,3,86,13,0.00,Sim,Ativo,N,A
53,1,108,875.60,19231,3,51,B2BGA,N,N,N,N,ANAPOLIS,MINIMERCADO,3,86,13,0.00,Sim,Ativo,N,A
54,1,108,875.60,29580,3,51,B2BGA,N,N,N,N,ANAPOLIS,MINIMERCADO,3,86,13,0.00,Sim,Ativo,N,A


In [176]:
bins = [-100, 50, 100, 500, 1200, 5000, 10000, 50000, 100000, 1000000]
labels = ['nulo', 'baixissimo', 'baixo', 'medio', 'mediano', 'alto', 'altissimo', 'gigante', 'raro']
data_v3['grau_valor'] = pd.cut(data_v3['Total Pedido'], bins=bins, labels=labels)
pd.value_counts(data_v3.grau_valor)

grau_valor
mediano       92405
medio         57907
baixo         48976
alto          20273
altissimo      9436
nulo           5772
baixissimo     4207
gigante          84
raro              0
Name: count, dtype: int64

In [177]:
bins_pedido = [-100, 50, 100, 500, 1200, 5000, 10000, 50000, 100000, 1000000]
labels_pedido = ['nulo', 'baixissimo', 'baixo', 'medio', 'mediano', 'alto', 'altissimo', 'gigante', 'raro']
data_v3['grau_valor'] = pd.cut(data_v3['Total Pedido'], bins=bins_pedido, labels=labels_pedido)
pd.value_counts(data_v3.grau_valor)

grau_valor
mediano       92405
medio         57907
baixo         48976
alto          20273
altissimo      9436
nulo           5772
baixissimo     4207
gigante          84
raro              0
Name: count, dtype: int64

In [178]:
data_v3['Prazo Atual'].value_counts()

Prazo Atual
0     83515
7     56507
10    39384
13    15633
5     12899
2     12449
1     10023
15     2358
17     2277
3      1970
12     1203
6       615
4       134
30       83
28       10
Name: count, dtype: int64

In [179]:
bins = [-10, 0, 20, 40, 60, 80, 100]
labels = ['muitoruim','baixo', 'medio', 'bom', 'muitobom', 'excelente']
data_v3['point_score'] = pd.cut(data_v3['Pontuação Score'], bins=bins, labels=labels)
pd.value_counts(data_v3.point_score)

point_score
excelente    170730
muitobom      64074
bom            3869
medio           387
baixo             0
muitoruim         0
Name: count, dtype: int64

In [180]:
bins = [-1 ,0, 1, 5, 10, 15, 30]
labels = ['avista', 'prazopequeno', 'prazonormal', 'prazomedio', 'altoprazo', 'mensal']
data_v3['prazo_status'] = pd.cut(data_v3['Prazo Atual'], bins=bins, labels=labels)
pd.value_counts(data_v3.prazo_status)

prazo_status
prazomedio      96506
avista          83515
prazonormal     27452
altoprazo       19194
prazopequeno    10023
mensal           2370
Name: count, dtype: int64

In [181]:
data_v3 = data_v3.drop(columns=[
 'Pontuação Score',
 'Prazo Atual',
 'Total Pedido'
 ])

In [182]:
data_v3.head(50)

,Operacao,Setor,Produto,Codigo GV,Tipo Movimento,Origem Pedido,Fora de Rota,Desconto na Rota,Desconto Fora de Rota,Pedido Exceção,Cidade,Categoria,Classe Risco Atual,Atraso Médio,Possui Comodato,Status do PDV,Negativado,Status,grau_valor,point_score,prazo_status
23,1,206,1898,2,51,B2BG,N,N,N,N,ABADIANIA,DEPOSITO DE BEBIDAS,5,6.00,Nao,Ativo,N,A,medio,muitobom,avista
51,1,108,9092,3,51,B2BGA,N,N,N,N,ANAPOLIS,MINIMERCADO,3,0.00,Sim,Ativo,N,A,medio,excelente,altoprazo
52,1,108,13065,3,51,B2BGA,N,N,N,N,ANAPOLIS,MINIMERCADO,3,0.00,Sim,Ativo,N,A,medio,excelente,altoprazo
53,1,108,19231,3,51,B2BGA,N,N,N,N,ANAPOLIS,MINIMERCADO,3,0.00,Sim,Ativo,N,A,medio,excelente,altoprazo
54,1,108,29580,3,51,B2BGA,N,N,N,N,ANAPOLIS,MINIMERCADO,3,0.00,Sim,Ativo,N,A,medio,excelente,altoprazo
55,1,108,32349,3,51,B2BG,N,N,N,N,ANAPOLIS,MINIMERCADO,3,0.00,Sim,Ativo,N,A,mediano,excelente,altoprazo
56,1,108,9092,3,51,B2BG,N,N,N,N,ANAPOLIS,MINIMERCADO,3,0.00,Sim,Ativo,N,A,mediano,excelente,altoprazo
57,1,108,988,3,51,B2BG,N,N,N,N,ANAPOLIS,MINIMERCADO,3,0.00,Sim,Ativo,N,A,mediano,excelente,altoprazo
58,1,108,18780,3,51,B2BG,N,N,N,N,ANAPOLIS,MINIMERCADO,3,0.00,Sim,Ativo,N,A,mediano,excelente,altoprazo
59,1,108,31064,3,51,B2BG,N,N,N,N,ANAPOLIS,MINIMERCADO,3,0.00,Sim,Ativo,N,A,mediano,excelente,altoprazo
